# 12-create-flat-file
This notebook takes all the cleaned CIN Census and merges them into a unique excel csv where each row is a date event (referral, CIN start, CIN end, etc.) with demographics info and a few relevant info to the event.
This means that there are a LOT of columns, potentially.

In [ ]:
import glob
import os
import yaml
from wrangling.log.cin_log import build_cinrecord
from wrangling.log.log import build_log

%run "00-config.ipynb"
%load_ext autoreload
%autoreload 2

### Config

In [ ]:
localauthority = 'Waltham Forest' # Write name of LA - this must match the folder name

In [ ]:
# Check there are CIN files in this folder
cin_files = glob.glob(os.path.join(cin_cleaned, localauthority, "*.xml"))
if len(cin_files)>0:
    print("Found {} clean CIN files for {}".format(len(cin_files), localauthority))
else:
    print("!!! No CIN files for {}".format(localauthority))

### 1. Run programme (takes a few minutes depending on quantity of data)

In [ ]:
# Create CIN record
cin = build_cinrecord(cin_files)

### 2. Quick data checks - does it look sensible?

In [ ]:
# Print a few values - check they make sense

print(
    "\n{} unique Child IDs".format(len(cin.LAchildID.unique())),
    "\n{} first event".format(cin.Date.min()),
    "\n{} last event".format(cin.Date.max())
)

for col in ['Date', 'LAchildID', 'GenderCurrent', 'Ethnicity', 'Disabilities', 'PersonBirthDate']:
    print("\n{} missing values for {}".format(cin[col].isna().sum(), col))

In [ ]:
# Look at event distribution
cin["Date"] = cin["Date"].astype("datetime64")
cin.groupby(cin["Date"].dt.year)['Date'].count().plot(kind="bar", figsize=(10,7))

### 3. Save to csv

In [ ]:
# Add column with LA name
cin['LA'] = localauthority

# Save
cin.to_csv(os.path.join(flatfile_folder, "{}_flatcin.csv".format(localauthority)), index=False)